# Link Prediction
In this file we will give you a simple example to show how to use AutoGL to do link prediction.

## Import libraries
First, you should import some libraries and you can set the random seed before you train the model.

In [1]:
from autogl.datasets import build_dataset_from_name
from autogl.solver import AutoLinkPredictor
from autogl.solver.utils import set_seed
import argparse
from autogl.backend import DependentBackend
from autogl.datasets.utils import split_edges
from autogl.module.train.evaluation import Auc
import yaml
import random
import torch
import numpy as np
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
set_seed(202106)

/Users/nargesalavisamani/opt/anaconda3/envs/gnas/lib/python3.8/site-packages/torch_geometric/typing.py:63: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: dlopen(/Users/nargesalavisamani/opt/anaconda3/envs/gnas/lib/python3.8/site-packages/torch_scatter/_version_cpu.so, 0x0006): tried: '/Users/nargesalavisamani/opt/anaconda3/envs/gnas/lib/python3.8/site-packages/torch_scatter/_version_cpu.so' (mach-o file, but is an incompatible architecture (have (arm64), need (x86_64)))
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/Users/nargesalavisamani/opt/anaconda3/envs/gnas/lib/python3.8/site-packages/torch_geometric/typing.py:74: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: dlopen(/Users/nargesalavisamani/opt/anaconda3/envs/gnas/lib/python3.8/site-packages/torch_cluster/_version_cpu.so, 0x0006): tried: '/Users/nargesalavisamani/opt/anaconda3/envs/gnas/lib/python3.8/site-

OSError: dlopen(/Users/nargesalavisamani/opt/anaconda3/envs/gnas/lib/python3.8/site-packages/torch_cluster/_version_cpu.so, 0x0006): tried: '/Users/nargesalavisamani/opt/anaconda3/envs/gnas/lib/python3.8/site-packages/torch_cluster/_version_cpu.so' (mach-o file, but is an incompatible architecture (have (arm64), need (x86_64)))

## Config & Dataset
Then, you can load dataset and graph nas methods from configs

In [19]:
def parse(args=None):
    parser = ArgumentParser("auto link prediction", formatter_class=ArgumentDefaultsHelpFormatter)
    parser.add_argument('--config', type=str, default="../../configs/lp_gcn_benchmark.yml")
    parser.add_argument('--dataset', choices=['cora', 'citeseer', 'pubmed'], default='cora', type=str)
    parser.add_argument("--hpo", type=str, default="tpe", help="hpo methods")
    parser.add_argument("--max_eval", type=int, default=50, help="max hpo evaluation times")
    parser.add_argument("--seed", type=int, default=0, help="random seed")
    parser.add_argument("--device", default=0, type=int, help="GPU device")
    args = parser.parse_args(args)
    return args

args = parse('')

dataset = build_dataset_from_name(args.dataset)
dataset = split_edges(dataset, 0.8, 0.05) # split the edges for dataset
if DependentBackend.is_dgl(): # add self-loop
    import dgl
    # add self loop to 0
    data = list(dataset[0])
    data[0] = dgl.add_self_loop(data[0])
    dataset = [data]
configs = yaml.load(open(args.config, "r").read(), Loader=yaml.FullLoader)
configs["hpo"]["name"] = args.hpo
configs["hpo"]["max_evals"] = args.max_eval
solver = AutoLinkPredictor.from_config(configs)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


[2022-11-09 23:02:49] WARNING (root/MainThread) Ignore passed dec since enc is a whole model


## Run 

In [21]:
solver.fit(dataset, time_limit=3600, evaluation_method=[Auc], seed=args.seed)
solver.get_leaderboard().show()
auc = solver.evaluate(metric="auc")
print("test auc: {:.4f}".format(auc))

INFO:LinkPredictor:Use the default train/val/test ratio in given dataset


[2022-11-09 23:03:03] INFO (LinkPredictor/MainThread) Use the default train/val/test ratio in given dataset
HPO Search Phase:



  0%|          | 0/50 [00:00<?, ?it/s]INFO:hyperopt.tpe:tpe_transform took 0.218073 seconds


[2022-11-09 23:03:03] INFO (hyperopt.tpe/MainThread) tpe_transform took 0.218073 seconds


INFO:hyperopt.tpe:TPE using 0 trials


...

INFO:hyperopt.tpe:TPE using 7/7 trials with best loss -0.953831


[2022-11-09 23:03:42] INFO (hyperopt.tpe/MainThread) TPE using 7/7 trials with best loss -0.953831


[2022-11-09 23:03:42] WARNING (root/MainThread) Ignore passed dec since enc is a whole model


 14%|█▍        | 7/50 [00:40<04:08,  5.78s/it]


KeyboardInterrupt: 